In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
#Hyperparameter setting
num_epochs = 10
batch_size = 4
learning_rate = 0.001


In [6]:
#Transforms
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

#Loading the datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Implementing the Conv Net

In [34]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self, X):
        X = self.pool(F.relu(self.conv1(X)))
        X = self.pool(F.relu(self.conv2(X)))
        X = X.reshape(-1,16*5*5)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
        return X
        
    
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_batches = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        images = images.to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        #Prompt
        if (i+1)%2000==0:
            print(f'epoch {epoch}/{num_epochs}, batch {i+1}/{n_batches}, loss = {loss.item():.5f}')
            
            
print('Training completed')       

epoch 0/10, batch 2000/12500, loss = 2.31711
epoch 0/10, batch 4000/12500, loss = 2.30696
epoch 0/10, batch 6000/12500, loss = 2.30231
epoch 0/10, batch 8000/12500, loss = 2.31160
epoch 0/10, batch 10000/12500, loss = 2.28927
epoch 0/10, batch 12000/12500, loss = 1.96262
epoch 1/10, batch 2000/12500, loss = 1.70053
epoch 1/10, batch 4000/12500, loss = 2.28046
epoch 1/10, batch 6000/12500, loss = 2.18886
epoch 1/10, batch 8000/12500, loss = 1.64887
epoch 1/10, batch 10000/12500, loss = 2.38928
epoch 1/10, batch 12000/12500, loss = 2.27653
epoch 2/10, batch 2000/12500, loss = 1.81013
epoch 2/10, batch 4000/12500, loss = 1.82880
epoch 2/10, batch 6000/12500, loss = 1.18706
epoch 2/10, batch 8000/12500, loss = 1.17925
epoch 2/10, batch 10000/12500, loss = 1.64641
epoch 2/10, batch 12000/12500, loss = 1.23424
epoch 3/10, batch 2000/12500, loss = 1.35684
epoch 3/10, batch 4000/12500, loss = 1.84802
epoch 3/10, batch 6000/12500, loss = 1.78923
epoch 3/10, batch 8000/12500, loss = 1.68532
epoc

### Calculating the accuracy

In [32]:
with torch.no_grad():
    n_total = 0
    n_correct = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        predictions = model(images)
        _, cls = torch.max(predictions, 1)
        
        n_total += labels.shape[0]
        n_correct += (cls==labels).sum().item()
        
    print(f'Accuracy is {100*n_correct/n_total}%')
    

Accuracy is 57.38%
